Рассматриваемые модели:
- geowac_lemmas_none_fasttextskipgram_300_5_2020 — обученная на Geowac модель с леммами и функциональными словами
- geowac_tokens_none_fasttextskipgram_300_5_2020 — обученная на Geowac модель с токенами(!) и функциональными словами
- ruwikiruscorpora-func_upos_skipgram_300_5_2019 — обученная на НКРЯ и википедии модель с леммами и функциональными словами
- tayga-func_upos_skipgram_300_5_2019 — обученная на Тайге модель с леммами и функциональными словами

## Подготовка данных
- импорт библиотек и модулей
- загрузка [датасета](https://docs.google.com/spreadsheets/d/1KYY9mnUkNmgRoCl15MWYu4b516KNXrF2/edit#gid=2122820980), добавление колонки с выражением предлог+сущ (в перспективе что-то с глаголом?)
- функции, использующиеся в экспериментах

In [ ]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 59.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13707 sha256=a942b1f0538b0324826bd26c11d5fc8e2035f496601703fe13a7a0a53f1fcbda
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
import gensim
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
import os
import pandas as pd
from pymorphy2 import MorphAnalyzer
import re
import shutil
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
morph = MorphAnalyzer()

In [ ]:
df = pd.read_excel(r'/content/drive/MyDrive/dataset_probe.xlsx')
v_cols = []
for line in df["контексты"]:
    cl_line = re.sub(r'[^\w\s]','', line.lower())
    words = cl_line.split()
    if "в" in words:
        v_cols.append(f"в {words[words.index('в')+1]}")
df["в + сущ"] = v_cols

In [ ]:
def data_for_clustering_tokens(some_df, model): #для получения векторов предлог + форма слова
    senses_ = []
    cols_ = []
    vecs_ = []
    vecs_lem = []
    for sns, wordc in zip(some_df["значение"], some_df["в + сущ"]):
        prep, noun = wordc.split()
        pvec = model.word_vec(prep)
        nvec = model.word_vec(noun)
        nvec_lem = model.word_vec(morph.normal_forms(noun)[0])
        wordc_vec = pvec + nvec
        wordc_vec_lem = pvec + nvec_lem
        senses_.append(sns)
        cols_.append(wordc)
        vecs_.append(wordc_vec)  
        vecs_lem.append(wordc_vec_lem)
    X_ = np.array(vecs_)
    X_lem = np.array(vecs_lem)
    new_df = pd.DataFrame({"короткое обозначение": senses_,
                   "выражение": cols_,
                   "вектор пред+форма сущ": vecs_})
    return X_lem, X_, new_df

def data_for_clustering_lems(some_df, model): #для получения векторов предлог + лемма
    senses_ = []
    cols_ = []
    vecs_ = []
    vecs_form = []
    for sns, wordc in zip(some_df["значение"], some_df["в + сущ"]):
        prep, noun = wordc.split()
        if prep in model:
            pvec = model.word_vec(prep)
            # try:
            nvec_lem = model.word_vec(morph.normal_forms(noun)[0])
            # except:
            #     pass
        else:
            pvec = model.word_vec(f"{prep}_ADP")
            try:
                nvec_lem = model.word_vec(f"{morph.normal_forms(noun)[0]}_NOUN")
            except:
                pass
        wordc_vec_lem = pvec+nvec_lem
        senses_.append(sns)
        cols_.append(wordc)
        vecs_.append(wordc_vec_lem)   
    X_ = np.array(vecs_)
    new_df = pd.DataFrame({"значение": senses_,
                   "выражение": cols_,
                   "вектор пред+лемма сущ": vecs_})
    return X_, new_df

def kmeans_clustering(n, matrix, slcted_columns):
    kmeans = KMeans(n_clusters=n, init = "random", n_init = "auto", max_iter = 500, algorithm='elkan').fit(matrix)
    df_res = slcted_columns.copy()
    kclusters = kmeans.predict(matrix)
    return df_res, kclusters

def show_result(some_df, cat_name):
    return some_df.loc[some_df['значение'] == cat_name]
    
def visual(some_X, some_df, cluster_cat, title):
    xs = some_X[:,0]
    ys = some_X[:,1]    
    colors = ['darkred', 'royalblue', 'teal', 'purple'] #цвета выделяемых кластеров

    figure(figsize=(14, 9), dpi=80)
    for context, x, y, cluster in zip(some_df["значение"], xs, ys, some_df[cluster_cat]):
        plt.scatter(x, y, color = colors[cluster])
        plt.annotate(context, xy=(x, y), color = colors[cluster])
        plt.title(title)
    plt.show()

def visual_cl(some_X, some_df, title, color):
    xs = some_X[:,0]
    ys = some_X[:,1]    

    figure(figsize=(11, 6), dpi=80)
    for context, x, y in zip(some_df["выражение"], xs, ys):
        plt.scatter(x, y, color = color)
        plt.annotate(context, xy=(x, y))
        plt.title(title)
    plt.show()

## Модель на Geowac (токены и предлоги)

Наиболее перспективная модель, здесь же самый расширенный эксперимент:
- смотрим на вектора токенов
- смотрим на вектора лемм (можно сравнить с другими моделями)
- смотрим на то, как влияют падежи

In [ ]:
# !wget http://vectors.nlpl.eu/repository/20/214.zip
os.mkdir("geowac_tokens")
shutil.unpack_archive("/content/drive/MyDrive/214.zip", "/content/geowac_tokens")
geowac_tok_model = gensim.models.KeyedVectors.load('geowac_tokens/model.model')

In [ ]:
X_gt_lemmas, X_gt_tokens, data_with_vecs_gt = data_for_clustering_tokens(df, geowac_tok_model)

<ipython-input-4-a594a1d05305>:8: DeprecationWarning: Call to deprecated `word_vec` (Use get_vector instead).
  pvec = model.word_vec(prep)
<ipython-input-4-a594a1d05305>:9: DeprecationWarning: Call to deprecated `word_vec` (Use get_vector instead).
  nvec = model.word_vec(noun)
<ipython-input-4-a594a1d05305>:10: DeprecationWarning: Call to deprecated `word_vec` (Use get_vector instead).
  nvec_lem = model.word_vec(morph.normal_forms(noun)[0])


У нас есть две матрицы, подходящие для кластеризации. В одной представлены векторы "предлог + лемма", в другой — "предлог + форма существительного". Начнем с первой матрицы

In [ ]:
df_clusters_gt_lemmas, clusters_gt_lemmas = kmeans_clustering(4, X_gt_lemmas, data_with_vecs_gt[["выражение", "значение", "вектор пред+лемма сущ"]])
df_clusters_gt_lemmas["clusters_lems"] = clusters_gt_lemmas

In [ ]:
df_clusters_gt, clusters_gt = kmeans_clustering(4, X_gt_tokens, data_with_vecs_gt[["выражение", "значение", "вектор пред+форма сущ"]]) 

In [ ]:
all_sim_words = geowac_tok_model.most_similar(positive=["в"], topn=15) #близкие слова
all_sim_words

[('и', 0.6202954053878784),
 ('на', 0.5644969940185547),
 ('во', 0.5504961609840393),
 ('а', 0.5404972434043884),
 ('ответыв', 0.5274618268013),
 ('назадв', 0.5249723792076111),
 ('по', 0.5239967107772827),
 ('представительном', 0.5209869146347046),
 ('но', 0.5168805122375488),
 ('с', 0.5132656693458557),
 ('действительно', 0.510128378868103),
 ('причем', 0.5058313608169556),
 ('что', 0.4994252920150757),
 ('определенно', 0.49794137477874756),
 ('прин', 0.49748551845550537)]

## Модель на Geowac (леммы и предлоги)

In [ ]:
# !wget http://vectors.nlpl.eu/repository/20/213.zip
os.mkdir("geowac_lemmas")
shutil.unpack_archive("/content/drive/MyDrive/213.zip", "/content/geowac_lemmas")
geowac_lem_model = gensim.models.KeyedVectors.load('geowac_lemmas/model.model')

In [ ]:
X_gl, data_with_vecs_gl = data_for_clustering_lems(df, geowac_lem_model)

<ipython-input-9-19d709d85cf5>:33: DeprecationWarning: Call to deprecated `word_vec` (Use get_vector instead).
  pvec = model.word_vec(prep)
<ipython-input-9-19d709d85cf5>:35: DeprecationWarning: Call to deprecated `word_vec` (Use get_vector instead).
  nvec_lem = model.word_vec(morph.normal_forms(noun)[0])


In [ ]:
df_clusters_gl, clusters_gl = kmeans_clustering(4, X_gl, data_with_vecs_gl[["выражение", "значение", "вектор пред+лемма сущ"]]) #переименовать колонку?
df_clusters_gl["clusters"] = clusters_gl

In [ ]:
all_sim_words = geowac_lem_model.most_similar(positive=["в"], topn=10) #близкие слова
all_sim_words

[('.вать', 0.7097395658493042),
 ('..вать', 0.6671736836433411),
 ('-вать', 0.6608783602714539),
 ('...вать', 0.6489165425300598),
 ('[вать', 0.6348358392715454),
 (',вать', 0.608148992061615),
 (':вать', 0.5987821221351624),
 ('нет.вать', 0.5923352837562561),
 ('и', 0.5908723473548889),
 ('с', 0.5821506977081299)]

## Модель на НКРЯ и википедии (леммы и предлоги)

In [ ]:
#загрузка модели с диска
os.mkdir("wiki")
shutil.unpack_archive("/content/drive/MyDrive/183.zip", "/content/wiki")
wiki_model = gensim.models.KeyedVectors.load_word2vec_format('wiki/model.bin',  binary=True)

In [ ]:
X_w, data_with_vecs_w = data_for_clustering_lems(df, wiki_model)

<ipython-input-21-b105dc88a533>:39: DeprecationWarning: Call to deprecated `word_vec` (Use get_vector instead).
  pvec = model.word_vec(f"{prep}_ADP")
<ipython-input-21-b105dc88a533>:41: DeprecationWarning: Call to deprecated `word_vec` (Use get_vector instead).
  nvec_lem = model.word_vec(f"{morph.normal_forms(noun)[0]}_NOUN")


In [ ]:
df_clusters_w, clusters_w = kmeans_clustering(4, X_w, data_with_vecs_w[["выражение", "значение", "вектор пред+лемма сущ"]])
df_clusters_w["clusters"] = clusters_w

In [ ]:
all_sim_words = wiki_model.most_similar(positive=["в_ADP"], topn=10) #близкие слова
all_sim_words

[(',_PUNCT', 0.5543971061706543),
 ('и_CCONJ', 0.5501202940940857),
 ('из_ADP', 0.516704797744751),
 ('..вать_VERB', 0.4881976842880249),
 ('._PUNCT', 0.48716896772384644),
 ('с_ADP', 0.4851657450199127),
 ('на_ADP', 0.48360520601272583),
 ('по_ADP', 0.4797956943511963),
 ('где_ADV', 0.4772920310497284),
 ('он_PRON', 0.47462034225463867)]

## Модель на Тайге (леммы и предлоги)

In [ ]:
#загрузка модели

!wget http://vectors.nlpl.eu/repository/20/186.zip
os.mkdir("taiga")
shutil.unpack_archive("/content/186.zip", "/content/taiga")
taiga_model = gensim.models.KeyedVectors.load_word2vec_format('taiga/model.bin',  binary=True)

--2023-05-27 08:41:10--  http://vectors.nlpl.eu/repository/20/186.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.181
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 640473374 (611M) [application/zip]
Saving to: ‘186.zip’

186.zip             100%[===================>] 610.80M   104MB/s    in 9.0s    

2023-05-27 08:41:19 (67.9 MB/s) - ‘186.zip’ saved [640473374/640473374]



In [ ]:
X_t, data_with_vecs_t = data_for_clustering_lems(df, taiga_model)

<ipython-input-21-b105dc88a533>:39: DeprecationWarning: Call to deprecated `word_vec` (Use get_vector instead).
  pvec = model.word_vec(f"{prep}_ADP")
<ipython-input-21-b105dc88a533>:41: DeprecationWarning: Call to deprecated `word_vec` (Use get_vector instead).
  nvec_lem = model.word_vec(f"{morph.normal_forms(noun)[0]}_NOUN")


In [ ]:
df_clusters_t, clusters_t = kmeans_clustering(4, X_w, data_with_vecs_w[["выражение", "значение", "вектор пред+лемма сущ"]]) 
df_clusters_t["clusters"] = clusters_t

In [ ]:
all_sim_words = taiga_model.most_similar(positive=["в_ADP"], topn=10) #близкие слова
all_sim_words

[('..в_VERB', 0.6442115902900696),
 ('.в_VERB', 0.6055962443351746),
 ('-в_VERB', 0.5981110334396362),
 (',в_VERB', 0.5870802998542786),
 ('.вать_VERB', 0.5721867680549622),
 ('..вать_VERB', 0.5599163770675659),
 ('.в_NOUN', 0.5222575068473816),
 ('...вать_VERB', 0.5144253969192505),
 ('...в_VERB', 0.5126088261604309),
 (',вать_VERB', 0.4979478120803833)]

## Таблица с кластерами

In [ ]:
df["Geowac_token_model — lemmas"] = clusters_gt_lemmas
df["Geowac_token_model — tokens"] = clusters_gt
df["Geowac_lemma_model"] = clusters_gl
df["WikiNCorpora_model"] = clusters_w
df["Taiga_model"] = clusters_t
df

,контексты,значение,в + сущ,Geowac_token_model — lemmas,Geowac_token_model — tokens,Geowac_lemma_model,WikiNCorpora_model,Taiga_model
0,"В универе очень интересно течет жизнь, постоя...",место,в универе,1,0,1,2,2
1,Растить в душе побег уныния -- преступление.,место,в душе,2,0,2,2,3
2,В Госдуме тоже пытаются бороться с курением,место,в госдуме,3,0,3,2,3
3,Я старалась подольше оставаться в саду,место,в саду,3,0,2,3,3
4,"Необходимо добавить, что в Китае банковский п...",место,в китае,3,0,1,3,3
...,...,...,...,...,...,...,...,...
95,Во время сильной грозы молния ударила в склад ...,Направление,в склад,3,3,2,3,3
96,"...многие хотят полететь в космос, но потом ос...",Направление,в космос,3,3,1,3,3
97,А на общем снимке его взгляд направлен не в ка...,Направление,в камеру,2,3,2,3,3
98,"Молодая девица смотрела в окно, она ждала весн...",Направление,в окно,2,3,2,3,3


In [ ]:
df.to_excel("w2v_clusterization_results.xlsx")